In [1]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY") 

# check the sever type according to API key
if openai.api_key.startswith("sk-"): # OpenAI
    server_type = "OpenAI"
else:
    server_type = "AzureOpenAI"

# check the enironment variable
print(f"sever_type: {server_type} | api_type: {openai.api_type}")


sever_type: OpenAI | api_type: open_ai


In [3]:
import requests

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}
data = {
    "model": "gpt-3.5-turbo-0613",
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"}
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    }
                },
                "required": ["location"]
            }
        }
    ]
}

response = requests.post(url, headers=headers, json=data)

In [5]:
response.json()

{'id': 'chatcmpl-7TXe18ljzlF6RVPOuoYHy8NtJgJOI',
 'object': 'chat.completion',
 'created': 1687275197,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Boston, MA"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 82, 'completion_tokens': 18, 'total_tokens': 100}}

In [12]:
response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Boston, MA"\n}'}}

In [5]:
def classify(input_string: str) -> str:
    functions = [{
        "name": "print_sentiment",
        "description": "A function that prints the given sentiment",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "enum": ["positive", "negative", "neutral"],
                    "description": "The sentiment."
                },
            },
            "required": ["sentiment"],
        }
    }]
    messages = [{"role": "user", "content": input_string}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call={"name": "print_sentiment"}
    )
    function_call = response.choices[0].message["function_call"]
    argument = json.loads(function_call["arguments"])
    return argument

In [7]:
response = classify("the product is bad")

In [8]:
response

{'sentiment': 'negative'}

In [2]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:./examples/data/Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass  `serpapi_api_key` as a named parameter. (type=value_error)

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")